### Coupled FEM-DEM Methods for Modeling Variably Saturated Granular Media


---


#### Chris Kees and Matthew Farthing

<a href="mailto:christopher.e.kees@usace.army.mil"> christopher.e.kees@usace.army.mil</a>

##### Coastal and Hydraulics Laboratory
##### US Army Engineer Research and Development Center


***SIAM CS&E, March 2, 2017***

<img src="./corps_logo.svg" alt="corps_logo" width="150">

### Acknowledgements
---
- Quezada de Luna, Song (ERDC)
- Bentley, Jenkins, Irvin (Clemson)
- Dimakopoulos, de Lataillade, and Cozzuto (HR Wallingford)
- Dawson (UT)
- Mattis (TU Munich)
- Myers (ARO)
- Guermond, Popov, Patty (TAMU)
- Sahni, Zhang, Shepherd, Ibanez (RPI): 
- Scovazzi (Duke)


### Application Contexts

### Vessel-wave-sea ice interactions
---
<img src="icebreaker25.png" alt="icebreaker" width="800">

### Variably Saturated Granular Media
---
<img src="leveeCartoon.svg" alt="leveeCartoon" width="800">

### Breakwater armoring
---
<img src="azoresBreakwaterWide.png" alt="azores" width="700">

### Multiphase Incompressible Flow
---

$$
\begin{eqnarray*}
  \nabla \cdot \mathbf v &=& 0 \\ 
  \frac{\partial \mathbf v }{\partial t} + \nabla \cdot \left[ \mathbf v \otimes \mathbf v -  \boldsymbol{\tau}(\phi)  \right] &=& \mathbf g - \frac{\nabla p}{\rho(\phi)}  \\
\frac{\partial \phi }{\partial t} + \nabla \mathbf v  \cdot \nabla \phi &=& 0 \\
\ldots
\end{eqnarray*}
$$

Many formulations possible. For this work see [Kees et al. 2011]

### Rigid Body Motion
---
$$
\begin{eqnarray}
\frac{d (m \mathbf{v}^c_s)}{d t} &=& \mathbf{F} \\
\frac{d (J_t \boldsymbol{\omega}_s)}{d t} &=& \mathbf{M}
\end{eqnarray}
$$
where
$$
\begin{eqnarray}
\mathbf{F} = mg + \int_{\partial \Omega_s} \boldsymbol{\sigma} \cdot \mathbf{n} dS + \mbox{contact force}\\
\mathbf{M} = \int_{\partial \Omega_s} (\mathbf{x} - \mathbf{x}^c) \times \boldsymbol{\sigma} \cdot \mathbf{n} dS + \mbox{contact moment}
\end{eqnarray}
$$
Where $\boldsymbol{\sigma} = p \mathbf{I} - \tau(\phi)$. For equations of motion and algorithms see [Tasora et al. 2008]


### Discrete Element Methods and LS-DEM
---
- Multi-body motion requires computing contacts
- If bodies collide, then the fluid domain topology changes  
- Complicated rigid body shapes create additional difficulties
- *The level set geometry description seems like a good tool*
- For *non-colliding*, *nice* geometries like ships we have adaptive ALE 
  - **Onkar Sahni, MS315, Fri,11:20** Mesh Motion and Adaptation for Two-Phase Flow Problems with Moving Objects,

See [Kawamoto etal. 2016] on LS-DEM

### Fluid-Solid Coupling (Jump Conditions)
---
- The fluid is viscous so $\mathbf{v} = \mathbf{v}_s$ along $\Omega_s$
- The fluid traction at the boundary is $\boldsymbol{\sigma} \cdot \mathbf{n}$
- We need to enforce a Dirichlet condition on fluid velocity
- We need to integrate the tractions to extract forces and moments for each particle

### Nitsche's Method (Weak Dirichlet Conditions)
---
- Integrate by parts on fluid momentum with no-slip only on $\Omega_s$:
$$\tiny \int_{\Omega_f} \left\{ \left[\frac{\partial \mathbf v }{\partial t} - \mathbf g + \frac{\nabla p}{\rho(\phi)}\right] \mathbf{w}  - \left[ \mathbf v \otimes \mathbf v -  \boldsymbol{\tau}(\phi)  \right] \cdot \nabla \mathbf w \right\}dV  - \int_{\partial \Omega_s} \boldsymbol{\nu} \frac{\partial \mathbf v}{\partial \mathbf n} \mathbf{w} dS = 0$$
- Nitsche 71 showed how to enforce Dirichlet conditions consistently via penalty:
$$\tiny \int_{\Omega_f} \left\{ \left[\frac{\partial \mathbf v }{\partial t} - \mathbf g + \frac{\nabla p}{\rho(\phi)}\right] \mathbf{w}  - \left[ \mathbf v \otimes \mathbf v -  \boldsymbol{\tau}(\phi)  \right] \cdot \nabla \mathbf w \right\} dV  - \int_{\partial \Omega_s} \left[ \nu(\phi) \frac{\partial \mathbf v}{\partial \mathbf n} \mathbf{w} + \nu(\phi)\frac{\partial \mathbf w}{\partial \mathbf n} \mathbf{v} + C(h_e)(\mathbf v-\mathbf v_s)\mathbf w\right]dS = 0$$


### Surface Integrals to Volume Integrals
---
- For now we avoid explicitly meshing $\partial \Omega_s$ by using the level set tools:
$$\int_{\partial \Omega_s} f dS = \int_{\Omega} \delta(\phi_s) f dV \approx \int_{\Omega} \delta_{\epsilon}(\phi_s) f dV$$

See [Osher and Fedkiw 2006]

### Perturbations of existing terms
---
$$ \tiny \int_{\Omega_f} \left[\frac{\partial \mathbf v }{\partial t} - \mathbf g + \frac{\nabla p}{\rho(\phi)} - \delta_{\epsilon}(\phi_s) \nu(\phi) \frac{\partial \mathbf v}{\partial \mathbf n} + \delta_{\epsilon}(\phi_s) C(h_e)(\mathbf v-\mathbf v_s)\right] \mathbf{w} dV - \int_{\Omega_f} \left[ \mathbf v \otimes \mathbf v -  \boldsymbol{\tau}(\phi)  - \delta_{\epsilon}(\phi_s)\nu(\phi) \mathbf{v} \otimes \mathbf{n} \right] \cdot \nabla \mathbf w dV  = 0$$

- Implementation is straightforward, in existing loop over elements, update existing element integrals
- At the same time, calculate $F_i$ and $M_i$ consistent with the Nitsche momentum flux

### Air/water + DEM solid (sequential splitting)
---
1. $\mathbf{v}^n,\phi^n \rightarrow \phi^{n+1}$  (air/water interface)
2. $\mathbf{v}^n,\phi^{n+1}, \phi_{s,i}^n p^n \rightarrow \mathbf{v}^*, \mathbf{F}^{n+1}_i, \mathbf{M}^{n+1}_i$ (fluid flow)
3. $\mathbf{v}^* \rightarrow \mathbf{v}^{n+1}$ (velocity correction)
4. $\mathbf{v}^{n+1},p^{n} \rightarrow p^{n+1}$ (pressure update)
5. $\mathbf{F}^{n+1}_i, \mathbf{M}^{n+1}_i \rightarrow \phi_{s,i}$ (solid dynamics)

1-4 are extensions of scheme from [Guermond and Salgado 2011]

### Example 1: Wall
---
<img src="dambreakib.gif" alt="dambreakib" width="700">

### Example 2: Flow around a cylinder
---
<img src="cylinder.gif" alt="cylinderib" width="700">

### Conclusions
---
- Pro's:
 - Easy to implement
 - Fast
 - Straightforward evaluation consisent forces and moments
 - Works directly from general signed distance representation
- Con's:
 - Probably $O(h)$
 - Requires resolution of solid boundary layer

### Roadmap
---
- Complete coupling to project Chrono DEM and ERDC's in-house DEM
- Investigate better smoothed/discrete Dirac functions, more analysis...
- Re-enable surface tension and contact line motion
- Thorough V&V suite for convergence and robustness
- Extend implementation and compare to more sophisticated methods (e.g from this session, XFEM)
- The code is open source https://github.com/erdc-cm/proteus
- Post-doc and student internships are available

### References

###### C. E. Kees, I. Akkerman, M. W. Farthing, and Y. Bazilevs. "A conservative level set method suitable for variable-order approximations and unstructured meshes". J. Comput. Phys. 230, 12 (June 2011), 4536-4558.
###### A. Tasora, D. Negrut, and M. Anitescu. "Large-scale parallel multi-body dynamics with frictional contact on the graphical processing unit." Proceedings of the Institution of Mechanical Engineers, Part K: Journal of Multi-body Dynamics 222, no. 4 (2008): 315-326.
###### R. Kawamoto, E. Ando, G. Viggiani, and J.E. Andrade, "Level set discrete element method for three-dimensional computations with triaxial case study"Journal of the Mechanics and Physics of Solids 91 (2016) 1–13
###### J.Nitsche. "Über ein Variationsprinzip zur Lösung von Dirichlet-Problemen bei Verwendung von Teilräumen, die keinen Randbedingungen unterworfen sind." Abhandlungen aus dem mathematischen Seminar der Universität Hamburg. Vol. 36. No. 1. Springer-Verlag, 1971.
###### J.L. Guermond and A.J. Salgado, 2011. "Error analysis of a fractional time-stepping technique for incompressible flows with variable density." SIAM Journal on Numerical Analysis, 49(3), pp.917-944.
###### S. Osher, R. Fedkiw. **Level set methods and dynamic implicit surfaces.** Springer Science & Business Media; 2006 Apr 6.